In [1]:
import pandas as pd
import numpy as np
#import geopandas as gpd
import urllib#pour récupérer les données
import bs4#pour rendre lisibles les données
import re
import time

Récupération des députés sur le site de l'Assemblée Nationale

In [44]:
def Scrap (url):
    req = urllib.request.Request(url)
    html = urllib.request.urlopen(req).read()
    page = bs4.BeautifulSoup(html, "lxml")
    return page

In [4]:
def ToDf (table) :
    Listgrossiere=pd.read_html(str(table))#on transforme le tableau en liste de dataframes
    listeDf=Listgrossiere[0]#on récupère le dataframe
    return listeDf

In [5]:
page=Scrap('http://www2.assemblee-nationale.fr/sycomore/resultats/(offset)/'+str(0)+'/(query)/IiBTRUxFQ1QgbV9jb2RlX2RlcHV0ZSwgaWRfYWN0ZXVyX3RyaWJ1biwgbGVnX21heF90cmlidW4sIG5vbSwgbm9tX2FmZmljaGUsIHByZW5vbSwgZGF0ZV9uYWlzLCBkYXRlX2RlY2VzIEZST00gZGVwdXRlIFdIRVJFIDE9MSAgQU5EIG1fY29kZV9kZXB1dGUgSU4gKFNFTEVDVCBtX2NvZGVfZGVwdXRlIEZST00gbWFuZGF0LCBkZXBhcnRlbWVudCAgV0hFUkUgbWFuZGF0Lm1fbnVtX2RlcCA9IGRlcGFydGVtZW50Lm1fbnVtX2RlcCBBTkQgbWFuZGF0Lm1fdHlwZV9tYW5kYXQgPSAxICkgT1JERVIgQlkgZGF0ZV9uYWlzIERFU0Mi')
table = page.find('table')
liste=ToDf(table)
for i in range(32):
    page=Scrap('http://www2.assemblee-nationale.fr/sycomore/resultats/(offset)/'+str((i+1)*500)+'/(query)/IiBTRUxFQ1QgbV9jb2RlX2RlcHV0ZSwgaWRfYWN0ZXVyX3RyaWJ1biwgbGVnX21heF90cmlidW4sIG5vbSwgbm9tX2FmZmljaGUsIHByZW5vbSwgZGF0ZV9uYWlzLCBkYXRlX2RlY2VzIEZST00gZGVwdXRlIFdIRVJFIDE9MSAgQU5EIG1fY29kZV9kZXB1dGUgSU4gKFNFTEVDVCBtX2NvZGVfZGVwdXRlIEZST00gbWFuZGF0LCBkZXBhcnRlbWVudCAgV0hFUkUgbWFuZGF0Lm1fbnVtX2RlcCA9IGRlcGFydGVtZW50Lm1fbnVtX2RlcCBBTkQgbWFuZGF0Lm1fdHlwZV9tYW5kYXQgPSAxICkgT1JERVIgQlkgZGF0ZV9uYWlzIERFU0Mi')
    table = page.find('table')#on récupère le seul tableau de la page qui liste des députés
    listeDf=ToDf(table)
    liste=pd.concat([liste,listeDf])#on récupère le dataframe et on l'ajoute à la liste
    
liste=liste.reset_index()#on recrée un index afin de ne pas s'arrêter à 500 puis recommencer comme dans les pages web
listeDepute=liste[['Nom','Né le']]#on récupère les variables qui nous intéressent  

In [15]:
listeDepute.to_csv(r'C:\Users\sylva\Desktop\git\Projet-Python-deputes\liste_depute.csv')#on met les données sur csv pour ne pas avoir à rescrapper à chaque fois

On s'intéresse ensuite à la récupération des mandats de chaque député (partie en cours).

In [6]:
def scrap_un_mandat(mandat):
    #On récupère et lave les titres des colonnes
    columns=mandat.findAll('dt')
    columns=re.sub('<[^<]+?>', '', str(columns))  
    
    #On récupère et lave les titres des lignes
    lines=mandat.findAll('dd')
    lines=re.sub('<[^<]+?>', '', str(lines))
    lines=re.sub('\t', '', str(lines))
    lines=re.sub("\n", '', str(lines))
    line=''
    
    #On met tout dans un dataframe
    columns=columns[1:-1].split(',')
    lines=lines[1:-1].split(',')
    df=pd.DataFrame([lines],columns=columns)
    return df

In [7]:
def RecupNom (page):
    nom_depute=page.find('title')#on va là où il y a le nom
    nom_depute=re.sub('<[^<]+?>', '', str(nom_depute))#on enlève les balises
    nom_depute=nom_depute.split()#on récupère les mots du titre
    mot=nom_depute[0]#on initialise au premier mot
    nom_dep=''#on initialise la variable où on va récupérer le nom
    while mot!='-' :#on prend les premiers mots qui composent le nom, avant la suite du titre
        nom_dep=nom_dep+' '+mot#on concatène le mot appartenant au nom avec les précédents
        nom_depute=nom_depute[1:]#on supprime le mot étudié
        mot=nom_depute[0]#pour passer au suivant
    return nom_dep#on retourne le mot complet. Il est surement possible de le faire beaucoup plus vite en regex, mais les miennes ne marchent pas.

In [8]:
tps1=time.time()
page=Scrap('http://www2.assemblee-nationale.fr/sycomore/fiche/(num_dept)/'+str(5))
listeMandat=page
liste=[]
numProbleme=[]
for i in range(5,19501) :
    if i%500==1 :#pour mesurer l'avancement du scrapping
        print(i)
    page=Scrap('http://www2.assemblee-nationale.fr/sycomore/fiche/(num_dept)/'+str(i+7))
    #Une petite blague c'est glissé dans le site : les mandats de sénateurs sont présent dans le code html mais ils sont "cachés"
    #On les enlève donc d'abord
    pouf=page.find('div', {'id':"mandats_an"})
    if pouf :
        depute=pouf.findAll('dl', {'class' : "deputes-liste-attributs sycomore"})
    else :#éviter les cas d'assemblée nationale constituante qui ont un autre nom que mandats_an
        depute=page.findAll('dl', {'class' : "deputes-liste-attributs sycomore"})
    for mandat in depute :
        try:
            df = scrap_un_mandat(mandat)
        except AssertionError :
            numProbleme.append(i)
        else :#ne marche pas en elif ! A cause du try
            df["nom du député"]=RecupNom(page)
            if len(liste)==0: #On initialise avec le premier mandat
                liste=df  
            else:
                liste=pd.concat([liste,df])#on fusionne les mandats
print(len(numProbleme))
tps2=time.time()
print("temps d'exécution",tps2-tps1)            

C:\Users\sylva\Anaconda3\lib\site-packages\ipykernel_launcher.py:27: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



501
1001
1501
2001
2501
3001
3501
4001
4501
5001
5501
6001
6501
7001
7501
8001
8501
9001
9501
10001
10501
11001
11501
12001
12501
13001
13501
14001
14501
15001
15501
16001
16501
17001
17501
18001
18501
19001
705
temps d'exécution 7518.672792196274


Il y des pages de députés jusqu'à 19500 sur le site.

In [34]:
liste=liste.rename(columns={' Département': 'Département', ' Groupe': 'Groupe', ' Législature': 'Législature', ' Mandat' :'Mandat'})

In [36]:
liste_dep=liste[['Département', 'Groupe', 'Législature', 'Mandat', 'Régime politique', 'nom du député']]

In [37]:
liste_dep.to_csv(r'C:\Users\sylva\Desktop\git\Projet-Python-deputes\liste_mandat.csv')
#on met les données sur csv pour ne pas avoir à rescrapper à chaque fois

In [ ]:
len(liste)/len(numProbleme)

La fonction de récupération suivante ne marche pas du tout (et est très longue)

In [52]:
list_dep=pd.read_csv((r'C:\Users\sylva\Desktop\git\Projet-Python-deputes\liste_depute.csv'))
compteur=0
for depute in list(list_dep['Nom']):
    dep_ute=re.sub(' ', '_', depute)
    dep_ute=re.sub(',', '', str(dep_ute))#pour les premiers et seconds prénoms
    try :
        text=Scrap('https://fr.wikipedia.org/wiki/'+dep_ute)   
    except : 
        UnicodeEncodeError#problème du aux i trémas (au moins)
    departement=text.find('span', {'class':'wd_p19'})#on récupère le département de naissance 
    departement=re.sub('<[^<]+?>', '', str(departement))#on le nettoie
    
    if compteur==0 :#initialisation   
        dep_naissance=pd.DataFrame([depute, departement],['Nom', 'Département de naissance'])#on en fait un df        
    else :
        dep_transit=pd.DataFrame([departement, depute], ['Département de naissance', 'Nom' ])
        dep_naissance=pd.concat([dep_naissance, dep_transit])#on rajoute l'information aux autres
    compteur+=1#et on passe aux étapes suivantes
    if compteur%500==0 :
        print(compteur)


KeyboardInterrupt: 

In [53]:
dep_naissance

,0
Nom,Ludovic Pajot
Département de naissance,None
Département de naissance,Nom
None,Typhanie Degois
Département de naissance,Nom
None,Lénaïck Adam
Département de naissance,Nom
None,Pierre Henriet
Département de naissance,Nom
None,Robin Reda


In [ ]:
dep_naissance.to_csv(r'C:\Users\sylva\Desktop\git\Projet-Python-deputes\dep_naissance.csv')
#on met les données sur csv pour ne pas avoir à rescrapper à chaque fois